# Preprocess

In [ ]:
import json
import numpy as np

# 定义输入文件名
input_file_path = 'trajectories-0500-0515.json'

# 初始化最大最小值
min_timestamp, max_timestamp = float('inf'), float('-inf')
min_position, max_position = float('inf'), float('-inf')

# 遍历文件以找到最大最小值
with open(input_file_path, 'r') as file:
    for line in file:
        record = json.loads(line)
        timestamps = np.array(record['timestamp']) / 1000  # 将时间戳从毫秒转换为秒
        local_y_positions = np.array(record['local_y']) * 0.3048  # 将位置从英尺转换为米
        land_id = record['lane_id']

        # 更新最大最小值
        min_timestamp = min(min_timestamp, timestamps.min())
        max_timestamp = max(max_timestamp, timestamps.max())
        min_position = min(min_position, local_y_positions.min())
        max_position = max(max_position, local_y_positions.max())

# 打印出找到的最大最小值（可选）
print(f"Timestamp range: {min_timestamp} - {max_timestamp}")
print(f"Position range: {min_position} - {max_position}")

In [ ]:
def compute_speed(record):
    timestamps = np.array(record['timestamp']) / 1000
    local_y_positions = np.array(record['local_y']) * 0.3048
    
    # 确保数据按时间排序
    sorted_indices = np.argsort(timestamps)
    timestamps = timestamps[sorted_indices]
    local_y_positions = local_y_positions[sorted_indices]

    
    # 计算速度
    speed = np.diff(local_y_positions) / np.diff(timestamps)
    speed = np.append(speed[0], speed)  # 保持速度数组与位置数组长度一致
    speed *= 2.23694 # m/s转化成mph
    
    return timestamps, local_y_positions, speed

# Create grids

In [ ]:
def get_grid(min_x, max_x, min_t, max_t, x_res, t_res):
    x_grid = np.arange(min_x, max_x + x_res, x_res)
    t_grid = np.arange(min_t, max_t + t_res, t_res)
    return x_grid, t_grid


x_res, t_res = 10, 0.8  # 分辨率可以根据需要调整
x_grid, t_grid = get_grid(min_position, max_position, min_timestamp, max_timestamp, x_res, t_res)

# Distribute the data into grids

In [ ]:
def grid_traj(record, x_grid, t_grid, global_x_t_speed_dict, global_x_t_density_dict):
    timestamps, positions, speeds = compute_speed(record)
    lane_ids = record['lane_id']
    
    t_grid_ind, data_ind = 0, 0
    while t_grid_ind < len(t_grid) and t_grid[t_grid_ind] < timestamps[0]:
        t_grid_ind += 1
    while t_grid_ind < len(t_grid):
        while timestamps[data_ind] < t_grid[t_grid_ind]:
            data_ind += 1
            if data_ind >= len(timestamps):
                break
        if data_ind >= len(timestamps):
            break
        for i in range(len(x_grid) - 1):
            if positions[data_ind] >= x_grid[i] and positions[data_ind] < x_grid[i + 1]:
                lane_id = lane_ids[data_ind]
                if lane_id not in global_x_t_speed_dict:
                    global_x_t_speed_dict[lane_id] = {}
                    global_x_t_density_dict[lane_id] = {}
                key = (x_grid[i], t_grid[t_grid_ind])
                if key not in global_x_t_speed_dict[lane_id]:
                    global_x_t_speed_dict[lane_id][key] = []
                    global_x_t_density_dict[lane_id][key] = 0
                global_x_t_speed_dict[lane_id][key].append(speeds[data_ind])
                global_x_t_density_dict[lane_id][key] += 1
        t_grid_ind += 1
    return

# 初始化全局速度和密度字典
global_x_t_speed_dict = {i: {} for i in range(1, 8)}
global_x_t_density_dict = {i: {} for i in range(1, 8)}

# 重新遍历文件，处理每一条数据
with open(input_file_path, 'r') as file:
    i = 0
    for line in file:
        record = json.loads(line)
        grid_traj(record, x_grid, t_grid, global_x_t_speed_dict, global_x_t_density_dict)
        i += 1
        if i % 200 == 0:
            print(f"Processed {i} vehicles")
    print("Complete!")

print(f"Length of global_x_t_speed_dict: {[len(global_x_t_speed_dict[i]) for i in range(1, 8)]}")

# Plot each vehicle

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker
import numpy as np
import datetime

def plot_all_lanes(title, x_res=10, t_res=2, plot_density=False):
    # 设置图像字体
    plt.rc('font', family='serif', size=14)
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(20, 20))
    axes = axes.flatten()

    jet = plt.cm.jet
    colors = [jet(x) for x in np.linspace(1, 0.5, 256)]
    green_to_red = LinearSegmentedColormap.from_list('GreenToRed', colors, N=256)
    
    for lane_id in range(1, 7):
        ax = axes[lane_id - 1]
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="3%", pad=0.05)

        if plot_density:
            data = np.array([[global_x_t_density_dict[lane_id].get((x, t), 0) for t in t_grid] for x in x_grid])
            label = 'Density'
            dmin, dmax = 0, np.max(data)
        else:
            data = np.array([[np.mean(global_x_t_speed_dict[lane_id].get((x, t), [0])) if global_x_t_speed_dict[lane_id].get((x, t)) else 0 for t in t_grid] for x in x_grid])
            label = 'Speed (mph)'
            #data = np.clip(data, 0, 55)
            dmin, dmax = 0, 55
            print(f"Lane {lane_id} clipped data min: {np.min(data)}, max: {np.max(data)}")  # 调试输出
        

            
        # 调整时间轴和位置轴的范围
        extent = [min_timestamp + 9 * 3600, max_timestamp + 9 * 3600, min_position, max_position]
        im = ax.imshow(data, cmap=green_to_red, aspect='auto', origin='lower', extent=extent, vmin = dmin, vmax = dmax)

        # 添加颜色条
        plt.colorbar(im, cax=cax).set_label(label, rotation=90, labelpad=20)
        ax.set_xlabel("Time")
        ax.set_ylabel("Meter Marker")
        ax.set_title(f"Lane {lane_id}")

         # 调整x轴时间格式
        ticks_loc = ax.get_xticks().tolist()
        ax.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
        x_datetime = [datetime.datetime.fromtimestamp(ts) for ts in ticks_loc]
        labels = [d.strftime('%H:%M:%S') for d in x_datetime]
        ax.set_xticklabels(labels, rotation=45)


    fig.suptitle(title)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# 调用绘图函数
plot_all_lanes('Vehicle Trajectories for All Lanes', x_res=10, t_res=0.8)